<a href="https://colab.research.google.com/github/AlvaroUriel/idal_ia3/blob/main/Ejercicio_opc_penguins.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#Importamos metodos a emplear para el proyecto
import numpy as np
import pandas as pd
from matplotlib.pyplot import figure
from matplotlib import pyplot as plt

from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from joblib import dump, load

In [4]:
import seaborn as sns
import pandas as pd
import plotly.express as px
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from math import log

In [5]:
dataset = sns.load_dataset("penguins")

In [6]:
dataset.head(10)

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,Male
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,Female
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,Female
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,Female
5,Adelie,Torgersen,39.3,20.6,190.0,3650.0,Male
6,Adelie,Torgersen,38.9,17.8,181.0,3625.0,Female
7,Adelie,Torgersen,39.2,19.6,195.0,4675.0,Male
8,Adelie,Torgersen,34.1,18.1,193.0,3475.0,NaN
9,Adelie,Torgersen,42.0,20.2,190.0,4250.0,NaN


In [8]:
#Comprobamos que los todos los datos son validos
contNull = dataset.isnull().sum()
contNan  = dataset.isna().sum()
sum(contNull, contNan)

species              19
island               19
bill_length_mm       21
bill_depth_mm        21
flipper_length_mm    21
body_mass_g          21
sex                  30
dtype: int64

In [9]:
dataset = sns.load_dataset("penguins")
dataset.dropna(inplace=True)
dataset.head(10)

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,Male
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,Female
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,Female
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,Female
5,Adelie,Torgersen,39.3,20.6,190.0,3650.0,Male
6,Adelie,Torgersen,38.9,17.8,181.0,3625.0,Female
7,Adelie,Torgersen,39.2,19.6,195.0,4675.0,Male
12,Adelie,Torgersen,41.1,17.6,182.0,3200.0,Female
13,Adelie,Torgersen,38.6,21.2,191.0,3800.0,Male
14,Adelie,Torgersen,34.6,21.1,198.0,4400.0,Male


In [10]:
#Comprobamos que los todos los datos son validos
contNull = dataset.isnull().sum()
contNan  = dataset.isna().sum()
sum(contNull, contNan)

species              0
island               0
bill_length_mm       0
bill_depth_mm        0
flipper_length_mm    0
body_mass_g          0
sex                  0
dtype: int64

In [18]:
dummies = pd.get_dummies(dataset.loc[:, ['species', 'island']])
numeric = dataset.select_dtypes('number')
X = pd.concat([dummies, numeric], axis=1)
Y = dataset.iloc[:,-1:]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3)


##Árbol de Decisión

In [19]:
model = DecisionTreeClassifier(criterion="entropy", max_depth=4, 
                               min_samples_split=5)
model.fit(X_train, Y_train)

DecisionTreeClassifier(criterion='entropy', max_depth=4, min_samples_split=5)

In [13]:
accuracy_score(Y_train, model.predict(X_train))

0.9184549356223176

In [16]:
model.score(X_test, Y_test)

0.87

In [47]:
#Transform into probabilities
new_probs = []
for value in Y_test['sex']:
  if value == "Male":
    new_probs.append(1.0)
  else:
    new_probs.append(0.0)

fpr, tpr, thresholds = roc_curve(new_probs, list(model.predict_proba(X_test)[:,1]), drop_intermediate=False)
df_roc = pd.DataFrame({'fpr':fpr, 'tpr':tpr, 'threshold':thresholds})

fig = px.line(df_roc, x='fpr', y='tpr', hover_data=['threshold'])
fig.show()    

##Regrsión Logística

In [52]:
modelRL = LogisticRegression(max_iter=300)
modelRL.fit(X_train, Y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



LogisticRegression(max_iter=300)

In [53]:
accuracy_score(Y_train, modelRL.predict(X_train))

0.8197424892703863

In [54]:
modelRL.score(X_test, Y_test)

0.88

In [55]:
#Transform into probabilities
new_probs = []
for value in Y_test['sex']:
  if value == "Male":
    new_probs.append(1.0)
  else:
    new_probs.append(0.0)

fpr, tpr, thresholds = roc_curve(new_probs, list(modelRL.predict_proba(X_test)[:,1]), drop_intermediate=False)
df_roc = pd.DataFrame({'fpr':fpr, 'tpr':tpr, 'threshold':thresholds})

fig = px.line(df_roc, x='fpr', y='tpr', hover_data=['threshold'])
fig.show()    

##Naive Bayes


In [23]:
from sklearn.naive_bayes import GaussianNB

In [24]:
modelNB = GaussianNB().fit(X_train, Y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



In [27]:
accuracy_score(Y_train, modelNB.predict(X_train))

0.7381974248927039

In [26]:
modelNB.score(X_test, Y_test)

0.73

In [56]:
#Transform into probabilities
new_probs = []
for value in Y_test['sex']:
  if value == "Male":
    new_probs.append(1.0)
  else:
    new_probs.append(0.0)

fpr, tpr, thresholds = roc_curve(new_probs, list(modelNB.predict_proba(X_test)[:,1]), drop_intermediate=False)
df_roc = pd.DataFrame({'fpr':fpr, 'tpr':tpr, 'threshold':thresholds})

fig = px.line(df_roc, x='fpr', y='tpr', hover_data=['threshold'])
fig.show()    

##Support Vector Machines

In [28]:
from sklearn.svm import SVC

In [63]:
modelSVC = SVC(kernel ='linear', probability=True).fit(X_train, Y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



In [64]:
accuracy_score(Y_train, modelSVC.predict(X_train))

0.9141630901287554

In [65]:
modelSVC.score(X_test, Y_test)

0.95

In [ ]:
modelSVC.pre

In [66]:
#Transform into probabilities
new_probs = []
for value in Y_test['sex']:
  if value == "Male":
    new_probs.append(1.0)
  else:
    new_probs.append(0.0)

fpr, tpr, thresholds = roc_curve(new_probs, list(modelSVC.predict_proba(X_test)[:,1]), drop_intermediate=False)
df_roc = pd.DataFrame({'fpr':fpr, 'tpr':tpr, 'threshold':thresholds})

fig = px.line(df_roc, x='fpr', y='tpr', hover_data=['threshold'])
fig.show()    